# Computing energies using SMIRNOFFEE

This example will show how the energy of paracetamol in multiple conformers can be evaluated using `smirnoffee` framework. 

We start by loading in the molecule of interest, in this case paracetamol as defined by its SMILES representation, using the [`openff-toolkit`](https://github.com/openforcefield/openff-toolkit) and generating a single conformer for it:

In [1]:
import openff.toolkit
import openff.units
import torch

molecule = openff.toolkit.Molecule.from_smiles("CC(=O)NC1=CC=C(C=C1)O")
molecule.generate_conformers(
    n_conformers=2, rms_cutoff=0.01 * openff.units.unit.angstrom
)

conformers = torch.tensor(
    [
        conformer.m_as(openff.units.unit.angstrom).tolist()
        for conformer in molecule.conformers
    ]
)
conformers.shape

torch.Size([2, 20, 3])

Conformers should either be tensors with a shape of `(n_atoms, 3)` or `(n_conformers, n_atoms, 3)`, and units of Å.

Next we will load in the force field that encodes the potential energy function we wish to evaluate and apply it to our molecule.

In [2]:
import openff.interchange

force_field = openff.toolkit.ForceField("openff_unconstrained-2.0.0.offxml")
interchange = openff.interchange.Interchange.from_smirnoff(
    force_field, molecule.to_topology()
)

In order to use an interchange object with this framework, we need to map it into a collection of tensors:

In [3]:
from smirnoffee.ff import convert_interchange

tensor_ff, [tensor_topology] = convert_interchange(interchange)

**Note:** The `convert_interchange` function can take either a single interchange object, or a list of multiple.

These tensors are returned as::

* a `smirnoffee.ff.TensorForceField`: this object stores the original values of the force field parameters.
* a list of ``smirnoffee.ff.TensorTopology``: each 'topology' will store for each handler matrices defining which parameters were assigned, as well as any virtual sites that were added.

Storing the parameter values separately from how they should be applied allows us to easily modify the values of the parameters and re-evaluate the energy using those parameters.

The energy of the molecule can then be directly be evaluated:

In [4]:
from smirnoffee.potentials import compute_energy

energies = compute_energy(tensor_topology.parameters, conformers, tensor_ff)

for conformer_idx, energy in enumerate(energies):
    print(f"conformer={conformer_idx} energy={energy.item():.3f} kJ / mol")

conformer=0 energy=-51.225 kJ / mol
conformer=1 energy=-55.653 kJ / mol
